In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import seaborn as sns

# data

In [2]:
npd = pd.read_csv('data/ANP2.2_NPD_data.csv',delimiter=';')
npd.head()

,NPD_ID,Noise Metric,Op Mode,Power Setting,L_200ft,L_400ft,L_630ft,L_1000ft,L_2000ft,L_4000ft,L_6300ft,L_10000ft,L_16000ft,L_25000ft
0,2CF650,EPNL,A,10000.0,106.2,101.1,97.2,92.5,84.2,75.0,68.0,61.4,53.4,43.3
1,2CF650,EPNL,A,25000.0,109.8,105.1,101.5,97.3,90.3,82.0,76.0,70.0,62.7,53.9
2,2CF650,EPNL,D,25000.0,109.8,105.1,101.5,97.3,90.3,82.0,76.0,70.0,62.7,53.9
3,2CF650,EPNL,D,40000.0,113.0,108.6,105.2,101.5,95.6,88.2,83.1,77.5,70.8,63.3
4,2CF650,LAmax,A,10000.0,99.2,91.9,86.7,81.0,72.1,63.0,56.7,49.6,41.6,33.1


In [4]:
df_sample_data = pd.read_csv('data/Sample_data1.csv')
df_sample_data.head()

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,TLM116-1664732580-schedule-0836,7,C,168,209,13.89892,100.59713,2022-10-04T17:23:06Z,A
1,TLM116-1664732580-schedule-0836,11,C,184,209,13.88681,100.59015,2022-10-04T17:23:22Z,A
2,TLM116-1664732580-schedule-0836,17,C,183,209,13.87550,100.58374,2022-10-04T17:23:38Z,A
3,TLM116-1664732580-schedule-0836,21,C,198,247,13.86586,100.57333,2022-10-04T17:23:54Z,A
4,TLM116-1664732580-schedule-0836,24,C,220,259,13.86200,100.55725,2022-10-04T17:24:10Z,A


In [5]:
df_sample_data['timestamp'] = pd.to_datetime(df_sample_data.timestamp)
df_sample_data.head()

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,TLM116-1664732580-schedule-0836,7,C,168,209,13.89892,100.59713,2022-10-04 17:23:06+00:00,A
1,TLM116-1664732580-schedule-0836,11,C,184,209,13.88681,100.59015,2022-10-04 17:23:22+00:00,A
2,TLM116-1664732580-schedule-0836,17,C,183,209,13.87550,100.58374,2022-10-04 17:23:38+00:00,A
3,TLM116-1664732580-schedule-0836,21,C,198,247,13.86586,100.57333,2022-10-04 17:23:54+00:00,A
4,TLM116-1664732580-schedule-0836,24,C,220,259,13.86200,100.55725,2022-10-04 17:24:10+00:00,A


In [8]:
# ------------- for export ------------- #
df_sample_data['new_date'] = [d.date() for d in df_sample_data['timestamp']]
df_sample_data['new_time'] = [d.time() for d in df_sample_data['timestamp']]
df_sample_data

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type,new_date,new_time
0,TLM116-1664732580-schedule-0836,7,C,168,209,13.89892,100.59713,2022-10-04 17:23:06+00:00,A,2022-10-04,17:23:06
1,TLM116-1664732580-schedule-0836,11,C,184,209,13.88681,100.59015,2022-10-04 17:23:22+00:00,A,2022-10-04,17:23:22
2,TLM116-1664732580-schedule-0836,17,C,183,209,13.87550,100.58374,2022-10-04 17:23:38+00:00,A,2022-10-04,17:23:38
3,TLM116-1664732580-schedule-0836,21,C,198,247,13.86586,100.57333,2022-10-04 17:23:54+00:00,A,2022-10-04,17:23:54
4,TLM116-1664732580-schedule-0836,24,C,220,259,13.86200,100.55725,2022-10-04 17:24:10+00:00,A,2022-10-04,17:24:10
...,...,...,...,...,...,...,...,...,...,...,...
302,TLM116-1664732580-schedule-0836,15,D,148,248,-6.10491,106.73810,2022-10-04 20:23:56+00:00,A,2022-10-04,20:23:56
303,TLM116-1664732580-schedule-0836,12,D,147,248,-6.10982,106.72575,2022-10-04 20:24:15+00:00,X,2022-10-04,20:24:15
304,TLM116-1664732580-schedule-0836,4,D,144,248,-6.12566,106.68590,2022-10-04 20:25:19+00:00,A,2022-10-04,20:25:19
305,TLM116-1664732580-schedule-0836,2,D,149,248,-6.12973,106.67585,2022-10-04 20:25:37+00:00,A,2022-10-04,20:25:37


In [15]:
# ------------- for export ------------- #
df_pamika = df_sample_data[['fa_flight_id','new_date','new_time','longitude','latitude','altitude','groundspeed']].copy()
df_pamika = df_pamika.rename(columns={'fa_flight_id':'name',
                              'new_date':'date',
                              'new_time':'time',
                              'longitude':'long',
                              'latitude':'lat',
                              'altitude':'altitude_ft',
                              'groundspeed':'speed'})
df_pamika['aircraft'] = 'AA' 
df_pamika.to_csv('pamika.csv')

In [5]:
df_departure_step = pd.read_csv('data/ANP2.2_Default_departure_procedural_steps.csv',delimiter=';')
df_departure_step = df_departure_step[df_departure_step.ACFT_ID == "737800"]
df_departure_step = df_departure_step[df_departure_step['Stage Length'] == "1"]
df_departure_step = df_departure_step[df_departure_step['Profile_ID'] == "DEFAULT"]
df_departure_step = df_departure_step.reset_index(drop=True)
df_departure_step

,ACFT_ID,Profile_ID,Stage Length,Step Number,Step Type,Thrust Rating,Flap_ID,End Point Altitude (ft),Rate Of Climb (ft/min),End Point CAS (kt),Accel Percentage (%)
0,737800,DEFAULT,1,1,Takeoff,MaxTakeoff,T_05,NaN,NaN,NaN,NaN
1,737800,DEFAULT,1,2,Climb,MaxTakeoff,T_05,1000.0,NaN,NaN,NaN
2,737800,DEFAULT,1,3,Accelerate,MaxTakeoff,T_05,NaN,1885.7,181.7,NaN
3,737800,DEFAULT,1,4,Accelerate,MaxTakeoff,T_01,NaN,2112.0,204.8,NaN
4,737800,DEFAULT,1,5,Climb,MaxTakeoff,T_00,2040.0,NaN,NaN,NaN
5,737800,DEFAULT,1,6,Climb,MaxClimb,T_00,3000.0,NaN,NaN,NaN
6,737800,DEFAULT,1,7,Accelerate,MaxClimb,T_00,NaN,1891.3,250.0,NaN
7,737800,DEFAULT,1,8,Climb,MaxClimb,T_00,5500.0,NaN,NaN,NaN
8,737800,DEFAULT,1,9,Climb,MaxClimb,T_00,7500.0,NaN,NaN,NaN
9,737800,DEFAULT,1,10,Climb,MaxClimb,T_00,10000.0,NaN,NaN,NaN


In [6]:
df_approach_step = pd.read_csv('data/Default_approach_procedural_steps.csv',delimiter=';')
df_approach_step = df_approach_step[df_approach_step.ACFT_ID == "737800"]
df_approach_step = df_approach_step[df_approach_step['Profile_ID'] == "DEFAULT"]
df_approach_step = df_approach_step.reset_index(drop=True)
df_approach_step

,ACFT_ID,Profile_ID,Step Number,Step Type,Flap_ID,Start Altitude(ft),Start CAS (kt),Descent Angle (deg),Touchdown Roll (ft),Distance (ft),Start Thrust
0,737800,DEFAULT,1,Descend-Idle,A_00,6000.0,248.9,3.0,NaN,NaN,NaN
1,737800,DEFAULT,2,Level-Idle,A_00,3000.0,249.5,NaN,NaN,25437.0,NaN
2,737800,DEFAULT,3,Level-Idle,A_01,3000.0,187.2,NaN,NaN,3671.0,NaN
3,737800,DEFAULT,4,Level-Idle,A_05,3000.0,174.7,NaN,NaN,5209.0,NaN
4,737800,DEFAULT,5,Descend-Idle,A_15,3000.0,151.4,3.0,NaN,NaN,NaN
5,737800,DEFAULT,6,Descend,A_30,2817.0,139.1,3.0,NaN,NaN,NaN
6,737800,DEFAULT,7,Land,A_30,NaN,NaN,NaN,393.8,NaN,NaN
7,737800,DEFAULT,8,Decelerate,A_30,NaN,139.0,NaN,NaN,3837.5,40.0
8,737800,DEFAULT,9,Decelerate,A_30,NaN,30.0,NaN,NaN,0.0,10.0


In [7]:
df_fix_point = pd.read_csv('data/ANP2.2_Default_fixed_point_profiles.csv',delimiter=';')
df_fix_point = df_fix_point[df_fix_point.ACFT_ID == "727200"]
df_fix_point = df_fix_point[df_fix_point['Stage Length'] == 1]
df_fix_point = df_fix_point.reset_index(drop=True)

df_fix_point


,ACFT_ID,Op Type,Profile_ID,Stage Length,Point Number,Distance (ft),Altitude AFE (ft),TAS (kt),Power Setting
0,727200,A,DEFAULT,1,1,-114487.0,6000.0,250.0,809.0
1,727200,A,DEFAULT,1,2,-57243.0,3000.0,142.0,4490.0
2,727200,A,DEFAULT,1,3,-28622.0,1500.0,142.0,4490.0
3,727200,A,DEFAULT,1,4,-19081.0,1000.0,142.0,4490.0
4,727200,A,DEFAULT,1,5,0.0,0.0,142.0,4490.0
5,727200,A,DEFAULT,1,6,406.0,0.0,142.0,4758.0
6,727200,A,DEFAULT,1,7,4060.0,0.0,30.0,1190.0
7,727200,D,DEFAULT,1,1,0.0,0.0,35.0,11895.0
8,727200,D,DEFAULT,1,2,5968.0,0.0,153.0,11895.0
9,727200,D,DEFAULT,1,3,13346.0,1000.0,153.0,11895.0


# create segment DF

In [8]:
# Create empty columns
df_sample_data_initial = pd.DataFrame(columns=['fa_flight_id',
                                               'altitude',
                                               'altitude_change',
                                               'groundspeed',
                                               'heading',
                                               'latitude',
                                               'longitude',
                                               'timestamp',
                                               'update_type'])
df_sample_data_airborne  = df_sample_data_initial.copy()
df_sample_data_reverse_final_approach  = df_sample_data_initial.copy()

# initial

In [9]:
# intitial
for index,row in df_sample_data.iterrows():
    if ((row['altitude'] <= df_departure_step['End Point Altitude (ft)']/100).iloc[-1]):
        df_sample_data_initial = df_sample_data_initial.append(row)
    else:
        break
    
df_sample_data_initial2 = df_sample_data_initial

In [10]:
nidx =np.arange(df_sample_data_initial2['timestamp'][0], df_sample_data_initial2['timestamp'].iloc[-1], 8000000 )
nidx = pd.to_datetime(nidx)

C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_21208\1622447894.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nidx =np.arange(df_sample_data_initial2['timestamp'][0], df_sample_data_initial2['timestamp'].iloc[-1], 8000000 )


In [11]:
df_sample_data_initial2['timestamp'] = df_sample_data_initial2['timestamp'].round('S').dt.tz_localize(None)
df_sample_data_initial2.set_index('timestamp', inplace=True)
df_sample_data_initial2.index.union(nidx)

DatetimeIndex(['2022-10-04 17:23:06', '2022-10-04 17:23:14',
               '2022-10-04 17:23:22', '2022-10-04 17:23:30',
               '2022-10-04 17:23:38', '2022-10-04 17:23:46',
               '2022-10-04 17:23:54', '2022-10-04 17:24:02',
               '2022-10-04 17:24:10', '2022-10-04 17:24:18',
               '2022-10-04 17:24:26', '2022-10-04 17:24:33',
               '2022-10-04 17:24:34', '2022-10-04 17:24:42',
               '2022-10-04 17:24:50', '2022-10-04 17:24:51',
               '2022-10-04 17:24:58', '2022-10-04 17:25:06',
               '2022-10-04 17:25:12', '2022-10-04 17:25:14',
               '2022-10-04 17:25:22', '2022-10-04 17:25:30',
               '2022-10-04 17:25:38', '2022-10-04 17:25:46',
               '2022-10-04 17:25:54', '2022-10-04 17:26:02',
               '2022-10-04 17:26:10', '2022-10-04 17:26:18',
               '2022-10-04 17:26:22', '2022-10-04 17:26:26',
               '2022-10-04 17:26:34', '2022-10-04 17:26:40',
               '2022-10-

In [12]:
res = df_sample_data_initial2.reindex(df_sample_data_initial2.index.union(nidx))
res = res[['altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]

In [13]:
res['altitude'] = pd.to_numeric(res['altitude'])
res['groundspeed'] = pd.to_numeric(res['groundspeed'])
res['heading'] = pd.to_numeric(res['heading'])

In [14]:
res = res.interpolate(method='time',limit_direction='both',limit=10)

In [15]:
i_departure = 1
res["segment"] =0

for index, row in res.iterrows():
    for i_departure in range(i_departure ,len(df_departure_step)):
        if row.altitude *100 <= df_departure_step.loc[i_departure]['End Point Altitude (ft)']:
            break
        elif row.groundspeed <= df_departure_step.loc[i_departure]['End Point CAS (kt)']:
            break
        
    res.loc[index,'segment']= i_departure
    
res

,altitude,groundspeed,heading,latitude,longitude,segment
2022-10-04 17:23:06,7.000000,168.000000,209.000000,13.898920,100.597130,1
2022-10-04 17:23:14,9.000000,176.000000,209.000000,13.892865,100.593640,1
2022-10-04 17:23:22,11.000000,184.000000,209.000000,13.886810,100.590150,3
2022-10-04 17:23:30,14.000000,183.500000,209.000000,13.881155,100.586945,3
2022-10-04 17:23:38,17.000000,183.000000,209.000000,13.875500,100.583740,3
2022-10-04 17:23:46,19.000000,190.500000,228.000000,13.870680,100.578535,3
2022-10-04 17:23:54,21.000000,198.000000,247.000000,13.865860,100.573330,3
2022-10-04 17:24:02,22.500000,209.000000,253.000000,13.863930,100.565290,5
2022-10-04 17:24:10,24.000000,220.000000,259.000000,13.862000,100.557250,5
2022-10-04 17:24:18,25.739130,231.130435,257.608696,13.860073,100.548666,5


In [16]:
res.reset_index(inplace=True)
res.rename(columns={'index':'timestamp'},inplace=True)
res

,timestamp,altitude,groundspeed,heading,latitude,longitude,segment
0,2022-10-04 17:23:06,7.000000,168.000000,209.000000,13.898920,100.597130,1
1,2022-10-04 17:23:14,9.000000,176.000000,209.000000,13.892865,100.593640,1
2,2022-10-04 17:23:22,11.000000,184.000000,209.000000,13.886810,100.590150,3
3,2022-10-04 17:23:30,14.000000,183.500000,209.000000,13.881155,100.586945,3
4,2022-10-04 17:23:38,17.000000,183.000000,209.000000,13.875500,100.583740,3
5,2022-10-04 17:23:46,19.000000,190.500000,228.000000,13.870680,100.578535,3
6,2022-10-04 17:23:54,21.000000,198.000000,247.000000,13.865860,100.573330,3
7,2022-10-04 17:24:02,22.500000,209.000000,253.000000,13.863930,100.565290,5
8,2022-10-04 17:24:10,24.000000,220.000000,259.000000,13.862000,100.557250,5
9,2022-10-04 17:24:18,25.739130,231.130435,257.608696,13.860073,100.548666,5


In [17]:
# res.to_csv('data/Sample_data2.csv')

# airborne

# final approach

In [18]:
# final approach
df_reverse_sample_data = df_sample_data.iloc[::-1]

for index, row in df_reverse_sample_data.iterrows():
    if ((row['altitude'] <= df_approach_step['Start Altitude(ft)']/100).iloc[0]):
        df_sample_data_reverse_final_approach = df_sample_data_reverse_final_approach.append(row)
    else:
        break
    
df_sample_data_final_approach = df_sample_data_reverse_final_approach.iloc[::-1]
df_sample_data_final_approach2 = df_sample_data_final_approach

In [19]:
i_departure = 0
df_sample_data_final_approach2["segment"] =0

for index, row in df_sample_data_final_approach2.iterrows():
    for i_departure in range(i_departure, len(df_approach_step)):
        if row.altitude *100 >= df_approach_step.loc[i_departure]['Start Altitude(ft)'] and row.groundspeed >= df_approach_step.loc[i_departure]['Start CAS (kt)']:
            break
        elif row.altitude * 100 >= df_approach_step.loc[i_departure]['Touchdown Roll (ft)']:
            break
        
    df_sample_data_final_approach2.loc[index,'segment']= i_departure
    
df_sample_data_final_approach2

C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_21208\3739199243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_data_final_approach2["segment"] =0
c:\Users\Nopnpr\anaconda3\envs\newenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type,segment
288,TLM116-1664732580-schedule-0836,60,D,243,139,-5.88652,106.81243,2022-10-04 20:17:48+00:00,A,2
289,TLM116-1664732580-schedule-0836,55,D,235,140,-5.91134,106.83418,2022-10-04 20:18:18+00:00,A,2
290,TLM116-1664732580-schedule-0836,51,D,220,140,-5.93522,106.85412,2022-10-04 20:18:48+00:00,A,2
291,TLM116-1664732580-schedule-0836,43,D,212,142,-5.95999,106.87405,2022-10-04 20:19:18+00:00,A,2
292,TLM116-1664732580-schedule-0836,40,D,204,162,-5.97605,106.88367,2022-10-04 20:19:38+00:00,A,2
293,TLM116-1664732580-schedule-0836,37,D,197,188,-5.99327,106.88490,2022-10-04 20:19:56+00:00,A,2
294,TLM116-1664732580-schedule-0836,35,D,188,203,-6.01041,106.87888,2022-10-04 20:20:18+00:00,A,2
295,TLM116-1664732580-schedule-0836,32,D,184,203,-6.03494,106.86856,2022-10-04 20:20:48+00:00,A,3
296,TLM116-1664732580-schedule-0836,31,D,182,203,-6.04926,106.86266,2022-10-04 20:21:07+00:00,A,3
297,TLM116-1664732580-schedule-0836,31,-,181,247,-6.05928,106.85247,2022-10-04 20:21:25+00:00,A,3


# interpolate

In [20]:
nidx =np.arange(df_sample_data_initial2['timestamp'][0], df_sample_data_initial2['timestamp'].iloc[-1], 8000000 )
nidx

KeyError: 'timestamp'

In [ ]:
nidx = pd.to_datetime(nidx)
type(nidx)

In [ ]:
df_sample_data_initial2['timestamp'] = df_sample_data_initial2['timestamp'].round('S').dt.tz_localize(None)
df_sample_data_initial2
# df_sample_data_initial2['timestamp'].loc[1]

In [ ]:
df_sample_data_initial2['timestamp'].dtypes

In [ ]:
# df_sample_data_initial2['timestamp'].astype('object')
df_sample_data_initial2.set_index('timestamp', inplace=True)
df_sample_data_initial2.head()

In [ ]:
df_sample_data_initial2.index.union(nidx)

In [ ]:
res = df_sample_data_initial2.reindex(df_sample_data_initial2.index.union(nidx))
res

In [ ]:
res = res[['altitude', 'groundspeed', 'heading', 'latitude', 'longitude','segment']]

In [ ]:
res.dtypes

In [ ]:
res['altitude'] = pd.to_numeric(res['altitude'])
res['groundspeed'] = pd.to_numeric(res['groundspeed'])
res['heading'] = pd.to_numeric(res['heading'])

In [ ]:
res

In [ ]:
res.dtypes

In [ ]:
res.head()

In [ ]:
res = res.interpolate(method='time',limit_direction='both',limit=10)
res

In [ ]:
res.isna().sum()

In [ ]:
# res.to_csv('data/Sample_data2.csv')

In [ ]:
res